# LightGBM

In [1]:
import pandas as pd
test_path = "/home/joseph/datasets/earthquake_damage_forecasting/test_data_embeds.csv"
test_df = pd.read_csv(test_path)
test_df.head()

,Unnamed: 0,building_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,...,geo_feat7,geo_feat8,geo_feat9,geo_feat10,geo_feat11,geo_feat12,geo_feat13,geo_feat14,geo_feat15,geo_feat16
0,0,300051,3,20,7,6,0,1,0,0,...,0.674943,1.233606,1.074826,0.948696,1.789101,-0.690246,1.215929,0.641817,1.489536,-1.748435
1,1,99355,2,25,13,5,0,1,0,0,...,-0.633836,0.383985,0.636826,0.662928,1.637369,-0.900778,1.354285,0.221195,0.662908,0.386138
2,2,890251,2,5,4,5,0,1,0,0,...,-0.570019,0.412793,1.512381,-0.302350,0.350699,-1.007432,1.000211,0.475425,-0.184000,-0.991295
3,3,745817,1,0,19,3,0,0,0,0,...,-1.826422,2.925866,2.126571,1.424021,2.839799,-1.503001,1.168797,2.822999,0.984964,0.840190
4,4,421793,3,15,8,7,0,1,0,0,...,-0.622701,1.714811,1.518751,0.280875,1.983836,-0.291633,1.130178,0.390867,1.355002,-1.796739


In [2]:
import numpy as np
def preprocess_embeds_df(df: pd.DataFrame) -> np.ndarray:
    """Assumes that df already contains the embeds. Processes the df and
    converts it to a numpy array to be used in self.predict.
    """
    if "building_id" in df.columns:
        df = df.drop(["building_id"], axis=1)

    if "Unnamed: 0" in df.columns:
        df = df.drop(["Unnamed: 0"], axis=1)

    return np.array(df)


In [3]:
test = preprocess_embeds_df(test_df)
test.shape

(86868, 81)

In [4]:
test[0][None].shape

(1, 81)

In [1]:
import lightgbm as lgb
model_path = "/home/joseph/Coding/ml_projects/earthquake_forecasting/models/lgb/model0.txt"
model = lgb.Booster(model_file=model_path)

In [6]:
model.num_feature()

81

In [11]:
model.feature_importance()

array([ 2298,  9425, 10043,  6319,   569,  1237,   438,   356,   634,
         776,  1235,   735,   582,   247,   322,  2078,  1008,   598,
         521,   200,    53,    68,   110,    41,    52,     0,   177,
         825,   307,   876,   141,   375,   964,   671,   536,  1334,
        1110,   535,   967,    78,   679,   697,   107,   693,  1337,
         395,  1068,   577,   150,  1380,  1074,    75,    76,   475,
          44,    59,    10,    54,   252,    49,   228,   254,   120,
         400,   164,  8098,  8318,  8108,  8257,  8212,  8333,  8066,
        7712,  8390,  8016,  7896,  7830,  7416,  7947,  7876,  7839],
      dtype=int32)

In [21]:
import multiprocessing
multiprocessing.cpu_count()

8

In [24]:
y_pred = model.predict(test[0][None], num_threads=1)
y_pred.shape

(1, 3)

In [28]:
from multiprocessing import Pool
import tqdm
pool = Pool(processes=8)
def predict(_id):
    print(f"Predicting for {_id}\n")
    y_pred = model.predict(test[0][None], num_threads=1)
    return _id

# starttime=time.time()
# pool = Pool(2)
# multiple_results = [pool.apply_async(functionToParallize) for i in [2,3]]
# p=[res.get() for res in multiple_results]
# print((time.time()-starttime)/60)

results = [pool.apply_async(predict, args=(x,)) for x in range(8)]
p = [res.get() for res in results]
print(p)
pool.close()
pool.join()

Predicting for 4
Predicting for 2
Predicting for 7
Predicting for 3
Predicting for 0
Predicting for 1

Predicting for 5

Predicting for 6


In [30]:
from multiprocessing import Pool
import tqdm
pool = Pool(processes=8)
def predict(_id):
    print(f"Predicting for {_id}\n")
    y_pred = model.predict(test[0][None], num_threads=1)
    return _id

results = [pool.apply(predict, args=(x,)) for x in range(8)]
print(results)
pool.close()
pool.join()

In [19]:
out

[]

In [ ]:
y_pred = model.predict(test[0][None])
y_pred.shape

In [16]:
# Probabilities for each class for a single sample
y_pred

array([[0.00084939, 0.26037801, 0.7387726 ]])

In [19]:
damage = y_pred.argmax(axis=1)+1
damage

array([3])

2 samples

In [20]:
y_pred = model.predict(test[:2])
y_pred.shape

(2, 3)

In [21]:
y_pred

array([[8.49388578e-04, 2.60378007e-01, 7.38772605e-01],
       [2.98472630e-03, 9.26326325e-01, 7.06889488e-02]])

In [22]:
damage = y_pred.argmax(axis=1)+1
damage

array([3, 2])